In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("train.csv", index_col=[0])

In [3]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [47]:
df["Weight"] = np.random.rand(len(df), 1)

df["bool"] = np.random.choice([False, True], size=len(df))

In [5]:
def weighted_avg(x, wts):
    return np.average(x, weights=wts, axis=0)

def weighted_std(x, wts):
    average = np.average(x, weights=wts, axis=0)
    xiance = np.average((x - average) ** 2, weights=wts, axis=0)
    return np.sqrt(xiance)

def weighted_skew(x, wts):
    return (np.average((x - weighted_avg(x, wts))**3, weights=wts, axis=0) /
            weighted_avg(x, wts)**(1.5))

def weighted_kurtosis(x, wts):
    return (np.average((x - weighted_avg(x, wts))**4, weights=wts, axis=0) /
            weighted_avg(x, wts)**(2))

In [6]:
funcs = [weighted_avg, weighted_std, weighted_skew, weighted_kurtosis]
metrics = ["LotArea", "LotFrontage", "SalePrice"]

In [9]:
df.dropna(subset=metrics, inplace=True)

In [53]:
def weighted_functions(df, funcs, metrics):
    names = [func.__name__ for func in funcs]
    groups = []
    
    for func in funcs:
        gb = df.groupby(["MSZoning", "bool"]).apply(lambda x: pd.Series(func(x[metrics], x["Weight"]), metrics)).T
        groups.append(gb)
        
    gb = df.groupby(["MSZoning", "SaleCondition"]).apply(lambda x: pd.Series(np.median(x[metrics]), metrics)).T
    groups.append(gb)
    names.append("median")
    
    return pd.concat(groups, axis=1, keys=names).round(2).T

In [54]:
res = weighted_functions(df, funcs, metrics=metrics)
res

LotArea  LotFrontage     SalePrice
                  MSZoning                                                 
weighted_avg      C (all)  False    8.169350e+03        70.37  6.908915e+04
                           True     8.483880e+03        59.98  8.356947e+04
                  FV       False    6.455910e+03        55.69  1.979872e+05
                           True     7.516810e+03        63.20  2.227690e+05
                  RH       False    6.059970e+03        63.16  1.442843e+05
                           True     6.998530e+03        54.44  1.350912e+05
                  RL       False    1.163017e+04        74.60  1.915904e+05
                           True     1.046026e+04        74.31  1.902426e+05
                  RM       False    6.820710e+03        54.64  1.279345e+05
                           True     6.554520e+03        52.68  1.224841e+05
weighted_std      C (all)  False    1.821140e+03        20.97  1.298888e+04
                           True     8.322200e+02         6.69  3.606028e+04
                  FV       False    3.130040e+03        23.70  4.627354e+04
                           True     2.482900e+03        18.56  4.679040e+04
                  RH       False    1.705650e+03        21.30  3.538687e+04
                           True     2.209450e+03         8.27  3.440222e+04
                  RL       False    1.362269e+04        25.68  8.586382e+04
                           True     3.713720e+03        19.91  8.481493e+04
                  RM       False    3.641890e+03        21.31  5.138672e+04
                           True     3.305610e+03        19.87  4.126444e+04
weighted_skew     C (all)  False    3.888175e+04        18.53 -1.558495e+05
                           True     6.547000e+02        -0.38 -3.665813e+04
                  FV       False    1.490547e+04         3.64  1.368068e+06
                           True    -1.571115e+04        -7.70  9.153222e+05
                  RH       False    1.517550e+03        -9.19 -7.178632e+05
                           True     3.349380e+03        -2.54  3.151087e+05
                  RL       False    2.371846e+07       102.73  1.376339e+07
                           True     1.014131e+05         9.52  1.449114e+07
                  RM       False    8.879113e+04         9.33  1.081475e+07
                           True     4.498334e+04        15.00  2.979566e+06
weighted_kurtosis C (all)  False    4.251857e+06       103.51  2.938853e+07
                           True     3.928682e+04         0.94  3.490628e+08
                  FV       False    3.765685e+06       151.68  4.545326e+08
                           True     1.483343e+06        76.70  4.100075e+08
                  RH       False    4.233795e+05        73.98  2.070675e+08
                           True     9.806564e+05         7.75  1.867026e+08
                  RL       False    4.326239e+10      2598.05  1.217543e+10
                           True     2.418344e+07       121.97  1.236486e+10
                  RM       False    2.043927e+07       200.63  9.062065e+09
                           True     1.066471e+07       290.90  1.961444e+09
median            C (all)  Abnorml  8.470000e+03      8470.00  8.470000e+03
                           Alloca   8.712000e+03      8712.00  8.712000e+03
                           Normal   9.770000e+03      9770.00  9.770000e+03
                  FV       Abnorml  3.392000e+03      3392.00  3.392000e+03
                           Normal   7.812500e+03      7812.50  7.812500e+03
                           Partial  8.125000e+03      8125.00  8.125000e+03
                  RH       Abnorml  7.800000e+03      7800.00  7.800000e+03
                           Normal   6.882500e+03      6882.50  6.882500e+03
                  RL       Abnorml  9.600000e+03      9600.00  9.600000e+03
                           AdjLand  8.285000e+03      8285.00  8.285000e+03
                           Alloca   7.018000e+03      7018.00  7.018000e+03
             